# Capítulo 12. Programando en la red

Alejandro E. Martínez Castro (amcastro@ugr.es)

Tomado de C. Severance. "Python for Informatics"

## Accediendo a datos de páginas web con *urllib*

En primer lugar importaremos el módulo ´urlopen´ de la librería ´urllib.request´ para acceder a un fichero que está en la red. Pruebe abrir el navegador y visualícelo [AQUI](http://www.py4inf.com/code/romeo.txt).

In [1]:
from urllib.request import urlopen

fhand = urlopen('http://www.py4inf.com/code/romeo.txt')

for line in fhand: 
    print(line.decode().strip())

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


A continuación podemos hacer un programa que acceda a los datos del fichero *'romeo.txt'* y devuelva la frecuencia de cada palabra. 

In [2]:
import urllib.request, urllib.parse, urllib.error

fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')
    
counts = dict()
for line in fhand:
    words = line.decode().split()
    for word in words:
        counts[word] = counts.get(word, 0) + 1
print(counts)

{'But': 1, 'soft': 1, 'what': 1, 'light': 1, 'through': 1, 'yonder': 1, 'window': 1, 'breaks': 1, 'It': 1, 'is': 3, 'the': 3, 'east': 1, 'and': 3, 'Juliet': 1, 'sun': 2, 'Arise': 1, 'fair': 1, 'kill': 1, 'envious': 1, 'moon': 1, 'Who': 1, 'already': 1, 'sick': 1, 'pale': 1, 'with': 1, 'grief': 1}


Lo que este bloque de código hace es: 
- Abre 'romeo.txt', y lo asigna a *fhand*. 
- Se genera un diccionario *counts*
- Línea a línea en *fhand*, extrae una lista con las palabras. Esto se realiza primero decodificando, y luego separando en una lista. 
- Finalmente, para cada línea, se cuenta el número de veces que aparece 

In [3]:
counts.get('But',0)

1

## Lectura de ficheros binarios con *urllib*

En ocasiones es necesario capturar ficheros que no contienen texto, como ficheros de imágenes, audio, vídeo, etc. En estos casos es posible realizar la descarga directa de este material, que está en la web, a nuestro disco duro. Para esto podemos usar también *urllib*. 

Para esto, el primer paso será abrir el contenido remoto, y descargarlo, mediante *read()*

In [4]:
import urllib.request, urllib.parse, urllib.error
img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg').read()
fhand = open('cover3.jpg', 'wb')
fhand.write(img)
fhand.close()

Est programa lee todo el fichero de una vez, y lo almacena en la variable *img*. Posteriormente abre un fichero, le asigna el nombre 'cover3.jpg', y vuelca el contenido de *img* en el fichero. Podemos verificar con el explorador de archivos que este fichero se ha creado en la carpeta de este cuaderno, y que contiene una imagen de portada de un libro. 

Si el fichero fuese muy grande (audio, película, etc), esta operación no puede realizarse de una vez, por limitaciones de memoria. En tal caso se recurre a un *buffer*. Mediante esta técnica, la información se compacta en bloques (buffers), que se van almacenando en el disco poco a poco, a medida que van leyéndose. 

In [5]:
import urllib.request, urllib.parse, urllib.error
img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg')
fhand = open('cover3.jpg', 'wb')
size = 0
while True:
  info = img.read(100000)
  if len(info) < 1: break
  size = size + len(info)
  fhand.write(info)

print(size, 'characters copied.')
fhand.close()

230210 characters copied.


En este ejemplo, se leen únicamente 100000 caracteres en cada bloque. Posteriormente se van volcando de forma aditiva al fichero 'cover3.jpg'. 

## Análisis HTML y scraping de la web

Uno de los usos habituales de *urllib* es para hacer **scraping de la web**. Esto no es otra cosa que escribir un programa que simule ser un navegador, capture datos de la web, y en vez de mostrarlos meramente en pantalla, realice otras tareas, como el análisis de esos datos. El propio uso de Google, por ejemplo, puede interpretarse como un scrapping de la web, ya que formalmente Google analiza los datos, enlaces a otras páginas, etc. 

## Análisis HTML usando expresiones regulares

Una de las formas más sencillas de extraer datos de la web consiste en usar las expresiones regulares, para búsqueda y extracción de cadenas de texto que encajan dentro de un patrón predefinido. 

Consideremos el siguiente bloque de código html: 

    <h1>La primera página</h1>
    <p>
    Si lo desea, puede ver más en el siguiente link
    <a href="http://www.dr-chuck.com/page2.htm">
    segunda página</a>.
    </p>
    
En este caso, podemos construir una expresión regular para extraer los enlaces, como sigue: 

    href="http[s]?://.+?"
    
Esta expresión regular busca en páginas que empiezan con "href=http://" o "href=https://", seguido de uno o más caracteres (.+?), seguido de una doble comilla. El signo de interrogación tras [s]? hace alusión a la búsqueda de http seguido de 0 o 1 vez el carácter 's'. Este tipo de búsqueda es *no ambiciosa* (no greedy), y trata de localizar siempre la mínima cadena de caracteres que cumple con el requisito de búsqueda.

Para extraer la parte deseada, debemos añadir paréntesis a nuestra expresión regular. 

El siguiente bloque de código pone esto en contexto. Al ejecutarlo, podemos introducir, sin comillas, cualquier nombre completa de web. Por ejemplo: 

    http://www.ugr.es
    
El código devuelve un listado de todos los links que enlaza esta página. 

In [6]:
# Búsqueda de links en una URL
import urllib.request, urllib.parse, urllib.error
import re
import ssl

# Ignorar errores de certificado SSL 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Enter - ')
html = urllib.request.urlopen(url).read()
links = re.findall(b'href="(http[s]?://.*?)"', html)
for link in links:
    print(link.decode())

Enter - http://www.ugr.es
http://www.ugr.es/
http://www.ugr.es/
https://www.ugr.es/sites/default/files/ugr-marca-horizontal-color%403x.png
https://www.facebook.com/universidadgranada/
https://twitter.com/canalugr
https://www.youtube.com/user/UGRmedios
https://www.ugr.es/servicios/correo-electronico
https://oficinavirtual.ugr.es/ai/
https://www.facebook.com/universidadgranada/
https://twitter.com/canalugr
https://www.youtube.com/user/UGRmedios
https://www.ugr.es/servicios/correo-electronico
https://oficinavirtual.ugr.es/ai/
https://sede.ugr.es
https://oficinavirtual.ugr.es
https://biblioteca.ugr.es
https://prado.ugr.es
https://directorio.ugr.es
https://transparente.ugr.es
https://calidad.ugr.es
https://calidad.ugr.es/politica
https://cartaservicios.ugr.es
https://secretariageneral.ugr.es/bougr/
https://archivo.ugr.es/
https://consejosocial.ugr.es/
https://defensor.ugr.es/
https://secretariageneral.ugr.es/pages/convenios
https://abierta.ugr.es
https://csirc.ugr.es
https://cic.ugr.es
http

Las expresiones regulares funcionan muy bien, siempre que el código HTML esté bien formateado. Pero esto, en la web, no es lo corriente. Existen muchas páginas con código "roto", etiquetas redundantes, etc. Es necesario por tanto recurrir a un analizador de código (que se denomina *parser*). El paquete BeautifulSoup ofrece un analizador avanzado. Veámoslo en la siguiente sección. 

## El paquete BeautifulSoup

Aunque el código HTML se parezca al XML (eXtensive Markup Languaje), la mayor parte de las páginas escritas en HTML tienen fallos en su sintaxis, etiquetado, etc. Si se introduce cualquier página HTML en un analizador (parser) de código XML, indicará que falla debido a estructura defectuosa. 

Existen paquetes Python dedicados a extraer información estructurada de código HTML. Uno de ellos es BeautifulSoup. 

En general, usaremos la librería *urllib* para leer la página, y *BeautifulSoup* para para extraer los atributos *href* de las etiquetas. 

El siguiente bloque de código pide al usuario que escriba una página web (completa). Entonces, abre la página, lee el contenido, y pasa los datos a un objeto de BeautifulSoup. Tras esto, genera todas las etiquetas de anclaje (de tipo <a)  e imprime el atributo de *href* para cada etiqueta. 


In [7]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignorar los errores del certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Introducir una URL completa, sin comillas, e.g. http://www.ugr.es
url = input('Enter - ')
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

# Recuperar todas las etiquetas de anclaje
tags = soup('a')
for tag in tags:
    print(tag.get('href', None))

Enter - http://www.ugr.es
#main-content
/
/en
https://www.facebook.com/universidadgranada/
https://twitter.com/canalugr
https://www.youtube.com/user/UGRmedios
https://www.ugr.es/servicios/correo-electronico
https://oficinavirtual.ugr.es/ai/
/
/
/en
https://www.facebook.com/universidadgranada/
https://twitter.com/canalugr
https://www.youtube.com/user/UGRmedios
https://www.ugr.es/servicios/correo-electronico
https://oficinavirtual.ugr.es/ai/
/
/en

/universidad/organizacion/saludo-rectora
/universidad/oficina-virtual
/universidad/sede-electronica
/historia/cronologia
/universidad/normativa/basica
/universidad/estudiar-en-la-ugr
/universidad/noticias/portada
/universidad/servicios

/estudiantes/accesibilidad
/estudiantes/acceso-a-la-universidad
/estudiantes/que-estudiar
/estudiantes/grados
/estudiantes/master-doctorados
/estudiantes/prado
/estudiantes/correo-electronico
/estudiantes/becas-y-ayudas
/estudiantes/practicas
/estudiantes/alojamiento/servicio-de-alojamiento
/estudiantes/movilid

Esta lista es algo más amplia que la que se obtuvo usando la expresión regular. Esto es así porque ahora podemos detectar enlaces que no incluyan los términos 'http'. 

Es posible usar BeutifulSoup para extraer varias partes de una etiqueta. 

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl

# Ignorar el certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Enter - ')
html = urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

# Extraer todas las etiquetas de anclaje
tags = soup('a')
for tag in tags:
    # Mirar en partes de una etiqueta
    print('TAG:', tag)
    print('URL:', tag.get('href', None))
    print('Contents:', tag.contents[0])
    print('Attrs:', tag.attrs)

Enter - http://www.ugr.es
TAG: <a class="visually-hidden focusable skip-link" href="#main-content">
      Pasar al contenido principal
    </a>
URL: #main-content
Contents: 
      Pasar al contenido principal
    
Attrs: {'href': '#main-content', 'class': ['visually-hidden', 'focusable', 'skip-link']}
TAG: <a class="is-active" href="/">Español</a>
URL: /
Contents: Español
Attrs: {'class': ['is-active'], 'href': '/'}
TAG: <a href="/en">English</a>
URL: /en
Contents: English
Attrs: {'href': '/en'}
TAG: <a class="facebook" href="https://www.facebook.com/universidadgranada/" target="_blank">facebook</a>
URL: https://www.facebook.com/universidadgranada/
Contents: facebook
Attrs: {'href': 'https://www.facebook.com/universidadgranada/', 'class': ['facebook'], 'target': '_blank'}
TAG: <a class="twitter" href="https://twitter.com/canalugr" target="_blank">twitter</a>
URL: https://twitter.com/canalugr
Contents: twitter
Attrs: {'href': 'https://twitter.com/canalugr', 'class': ['twitter'], 'target

IndexError: list index out of range

El analizador html.parser es el standard incluido en Python 3. Hay otros analizadores. Ver en: 

http://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser